<a href="https://colab.research.google.com/github/KWeatherwalks/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/Kevin_Weatherwalks_LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')
warnings.filterwarnings(action='ignore', category=FutureWarning, module='category_encoder')
warnings.filterwarnings(action='ignore', category=UserWarning, module='sklearn')
warnings.filterwarnings(action='ignore', category=RuntimeWarning, module='sklearn')

# I. Wrangle Data

In [2]:
import pandas as pd
import pandas_profiling
import numpy as np

def wrangle(filepath):
  # Read New York City property sales data
  df = pd.read_csv(filepath, parse_dates=['SALE DATE','YEAR BUILT'],
                   infer_datetime_format=True,
                   index_col='SALE DATE',
                   dtype={'BLOCK': object,
                          'LOT': object}
                   )

  # Change column names: replace spaces with underscores
  df.columns = [col.replace(' ', '_') for col in df]
  df.index.name = df.index.name.replace(' ', '_')


  # SALE_PRICE was read as strings.
  # Remove symbols, convert to integer
  df['SALE_PRICE'] = (
      df['SALE_PRICE']
      .str.replace('$','')
      .str.replace('-','')
      .str.replace(',','')
      .astype(int)
  )
  # LAND_SQUARE_FEET is a string
  # Remove symbols, empty strings, convert to float
  df['LAND_SQUARE_FEET'] = (
      df['LAND_SQUARE_FEET']
      .str.replace(',','')
      .str.replace('#','')
      .apply(lambda x: 0 if len(str(x))==0 else x)
      .astype(float)
  )

  # Subset data
  condition = (
      (df['BUILDING_CLASS_CATEGORY']=='01 ONE FAMILY DWELLINGS') &
      (df['SALE_PRICE'] > 100_000) & (df['SALE_PRICE']<2_000_000)
               )
  df = df[condition]

  # BOROUGH is a numeric column, but arguably should be a categorical feature,
  # so convert it from a number to a string
  df['BOROUGH'] = df['BOROUGH'].astype(str)

  # Reduce cardinality for NEIGHBORHOOD feature

  # Get a list of the top 10 neighborhoods
  top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

  # At locations where the neighborhood is NOT in the top 10, 
  # replace the neighborhood with 'OTHER'
  df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
  

  # Drop columns with null data
  df.drop(columns=['APARTMENT_NUMBER', 'EASE-MENT'], inplace=True)

  # Drop columns with too many categories
  drop_cols = [col for col in df.select_dtypes('object') if df[col].nunique() > 15]
  df.drop(columns=drop_cols, inplace=True)

  return df

In [3]:
FILE_PATH = DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv'
df = wrangle(FILE_PATH)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3151 entries, 2019-01-01 to 2019-04-30
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         3151 non-null   object 
 1   NEIGHBORHOOD                    3151 non-null   object 
 2   BUILDING_CLASS_CATEGORY         3151 non-null   object 
 3   TAX_CLASS_AT_PRESENT            3151 non-null   object 
 4   BUILDING_CLASS_AT_PRESENT       3151 non-null   object 
 5   ZIP_CODE                        3151 non-null   float64
 6   RESIDENTIAL_UNITS               3151 non-null   float64
 7   COMMERCIAL_UNITS                3151 non-null   float64
 8   TOTAL_UNITS                     3151 non-null   float64
 9   LAND_SQUARE_FEET                3151 non-null   float64
 10  GROSS_SQUARE_FEET               3151 non-null   float64
 11  TAX_CLASS_AT_TIME_OF_SALE       3151 non-null   int64  
 12  BUILDING_CLASS_A

In [5]:
df.select_dtypes(include='object').sample(3)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,BUILDING_CLASS_AT_TIME_OF_SALE
SALE_DATE,,,,,,
2019-01-02,5,OTHER,01 ONE FAMILY DWELLINGS,1,A2,A2
2019-02-27,3,OTHER,01 ONE FAMILY DWELLINGS,1,A1,A1
2019-04-11,4,OTHER,01 ONE FAMILY DWELLINGS,1,A1,A1


In [6]:
df.select_dtypes(include='object').nunique()

BOROUGH                            5
NEIGHBORHOOD                      11
BUILDING_CLASS_CATEGORY            1
TAX_CLASS_AT_PRESENT               2
BUILDING_CLASS_AT_PRESENT         13
BUILDING_CLASS_AT_TIME_OF_SALE    11
dtype: int64

In [7]:
df.select_dtypes(include='number').sample(3)

,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE_DATE,,,,,,,,
2019-03-12,11234.0,1.0,0.0,1.0,2000.0,1110.0,1,635000
2019-02-21,10310.0,1.0,0.0,1.0,7500.0,1260.0,1,625000
2019-01-11,10465.0,1.0,0.0,1.0,2500.0,2140.0,1,570000


# II. Split Data

In [8]:
# Split data into X matrix and y vector
target = 'SALE_PRICE'
y = df[target]
X = df.drop(columns=[target])

In [9]:
# Split data into training and test sets
train_mask = (df.index >= '2019-01') & (df.index < '2019-04')
test_mask = (df.index >= '2019-04') & (df.index < '2019-05')
X_train, y_train = X[train_mask], y[train_mask]
X_test, y_test = X[test_mask], y[test_mask]

# III. Establish Baseline

In [10]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_mean = [y_test.mean()] * len(y_test)
y_median = [y_test.median()] * len(y_test)

print('Mean Price MAE:', mean_absolute_error(y_test, y_mean))
print('Median Price MAE:', mean_absolute_error(y_test, y_median))

Mean Price MAE: 216602.40811504182
Median Price MAE: 210659.02639751552


# IV. Build Model

In [11]:
# Import classes
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from category_encoders import OneHotEncoder
from sklearn.feature_selection import SelectKBest

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [12]:
# One-hot encoding
ohe = OneHotEncoder(use_cat_names=True)
feature_selector = SelectKBest(k=2)

# Fit OHE to training data
ohe.fit(X_train)
ohe.fit(X_test)
# Transform data for feature selection
XT_train = ohe.transform(X_train)
XT_test = ohe.transform(X_test)

# Select k best features
feature_selector.fit(XT_train, y_train)

# Transform data for regression
XTT_train = feature_selector.transform(XT_train)
XTT_test = feature_selector.transform(XT_test)

In [13]:
XT_train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2507 entries, 2019-01-01 to 2019-03-30
Data columns (total 47 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   BOROUGH_2                                        2507 non-null   int64  
 1   BOROUGH_3                                        2507 non-null   int64  
 2   BOROUGH_4                                        2507 non-null   int64  
 3   BOROUGH_5                                        2507 non-null   int64  
 4   BOROUGH_1                                        2507 non-null   int64  
 5   NEIGHBORHOOD_OTHER                               2507 non-null   int64  
 6   NEIGHBORHOOD_FLUSHING-SOUTH                      2507 non-null   int64  
 7   NEIGHBORHOOD_QUEENS VILLAGE                      2507 non-null   int64  
 8   NEIGHBORHOOD_SO. JAMAICA-BAISLEY PARK            2507 non-null   int64  
 9   NEIGHBORHOOD

In [14]:
XTT_train.shape

(2507, 2)

In [15]:
# Build Regression Models
model_linear = LinearRegression()
model_ridge = Ridge(alpha=2.0)
model_ridgeCV = RidgeCV(alphas=(0.1,1.0,10.0))

# Fit models
model_linear.fit(XTT_train, y_train)
model_ridge.fit(XTT_train, y_train)
model_ridgeCV.fit(XTT_train, y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), cv=None, fit_intercept=True,
        gcv_mode=None, normalize=False, scoring=None, store_cv_values=False)

# V. Check Metrics

In [16]:
def display_score(text, score):
  score = round(score, 2)
  output = '{}:\t {:>13.2f}'.format(text, score)
  print(output)
  return None

In [18]:
# Baseline Stats
print(10*'-' + 'Baseline Metrics' + 10*'-')
print('Mean Price MAE:\t ', round(mean_absolute_error(y_test, y_mean),2))
print('Median Price MAE:', round(mean_absolute_error(y_test, y_median),2))
print()

print(10*'-' + 'Linear Regression Metrics' + 10*'-')
display_score('Train MAE', 
              mean_absolute_error(y_train, model_linear.predict(XTT_train)))
display_score('Test MAE', mean_absolute_error(y_test, 
                                              model_linear.predict(XTT_test)))
display_score('Train RMSE:', 
              mean_squared_error(y_train, 
                                 model_linear.predict(XTT_train),
                                 squared=False))
display_score('Test RMSE', 
              mean_squared_error(y_test, 
                                 model_linear.predict(XTT_test),
                                 squared=False))

print(10*'-' + 'Ridge Regression Metrics' + 10*'-')
display_score('Train MAE', 
              mean_absolute_error(y_train, 
                                  model_ridge.predict(XTT_train)))
display_score('Test MAE', 
              mean_absolute_error(y_test, 
                                  model_ridge.predict(XTT_test)))
display_score('Train RMSE', 
              mean_squared_error(y_train, 
                                 model_ridge.predict(XTT_train),
                                 squared=False))
display_score('Test RMSE', 
              mean_squared_error(y_test, 
                                 model_ridge.predict(XTT_test),
                                 squared=False))

print(10*'-' + 'RidgeCV Metrics' + 10*'-')
display_score('Train MAE', 
              mean_absolute_error(y_train, 
                                  model_ridgeCV.predict(XTT_train)))
display_score('Test MAE', 
              mean_absolute_error(y_test, 
                                  model_ridgeCV.predict(XTT_test)))
display_score('Train RMSE:', 
              mean_squared_error(y_train, 
                                 model_ridgeCV.predict(XTT_train),
                                 squared=False))
display_score('Test RMSE', 
              mean_squared_error(y_test, 
                                 model_ridgeCV.predict(XTT_test),
                                 squared=False))

----------Baseline Metrics----------
Mean Price MAE:	  216602.41
Median Price MAE: 210659.03

----------Linear Regression Metrics----------
Train MAE:	     209218.29
Test MAE:	     206567.01
Train RMSE::	     282587.20
Test RMSE:	     291771.72
----------Ridge Regression Metrics----------
Train MAE:	     209420.27
Test MAE:	     206731.46
Train RMSE:	     282615.63
Test RMSE:	     292226.32
----------RidgeCV Metrics----------
Train MAE:	     209331.53
Test MAE:	     206651.84
Train RMSE::	     282596.19
Test RMSE:	     292077.15


# VI. Tune Model

In [21]:
previous_features = set()
# Baseline metrics to compare to best model
min_MAE = min(mean_absolute_error(y_test, y_mean),
              mean_absolute_error(y_test, y_median))
min_RMSE = min(mean_squared_error(y_test, y_mean, squared=False),
               mean_squared_error(y_test, y_median, squared=False))
# Establish a best model
best_model = model_ridge
for num_features in range(1,len(XT_train.columns)+1):
  # Transform data
  feature_selector = SelectKBest(k=num_features)

  # Select k best features
  feature_selector.fit(XT_train, y_train)

  # Transform data for regression
  XTT_train = feature_selector.transform(XT_train)
  XTT_test = feature_selector.transform(XT_test)
  
  # Update features
  new_features = set(XT_train.columns[feature_selector.get_support()])
  added_features = new_features.difference(previous_features)
  previous_features = new_features


  # Fit model
  model = Ridge(alpha=2.0)
  model.fit(XTT_train, y_train)

  # Print results
  print(60*'-')
  print(10*'-' + f'Ridge Regression Metrics with {num_features} features' + 10*'-')
  print('Features added:\t', added_features)
  display_score('Train MAE', 
                mean_absolute_error(y_train, 
                                    model.predict(XTT_train)))
  display_score('Test MAE', 
                mean_absolute_error(y_test, 
                                    model.predict(XTT_test)))
  display_score('Train RMSE', 
                mean_squared_error(y_train, 
                                  model.predict(XTT_train),
                                  squared=False))
  display_score('Test RMSE', 
                mean_squared_error(y_test, 
                                  model.predict(XTT_test),
                                  squared=False))
  print(60*'-')


  # Update best model
  current_mae = mean_absolute_error(y_test, model.predict(XTT_test))
  current_rmse = mean_squared_error(y_test,
                                    model.predict(XTT_test), 
                                    squared=False)
  if current_rmse < min_RMSE:
    best_model = model
    best_features = new_features
    min_RMSE = current_rmse
  if current_mae < min_MAE:
    min_MAE = current_mae


------------------------------------------------------------
----------Ridge Regression Metrics with 1 features----------
Features added:	 {'BOROUGH_1'}
Train MAE:	     214702.48
Test MAE:	     211563.09
Train RMSE:	     291543.82
Test RMSE:	     298531.70
------------------------------------------------------------
------------------------------------------------------------
----------Ridge Regression Metrics with 2 features----------
Features added:	 {'BUILDING_CLASS_AT_TIME_OF_SALE_A3'}
Train MAE:	     209420.27
Test MAE:	     206731.46
Train RMSE:	     282615.63
Test RMSE:	     292226.32
------------------------------------------------------------
------------------------------------------------------------
----------Ridge Regression Metrics with 3 features----------
Features added:	 {'BUILDING_CLASS_AT_PRESENT_A3'}
Train MAE:	     209341.64
Test MAE:	     206652.33
Train RMSE:	     282598.38
Test RMSE:	     292170.41
------------------------------------------------------------
---

In [22]:
min_RMSE

232810.38391252127

# VII. Communicate Results


In [30]:
print(best_model.coef_)
print(best_model.intercept_)

[-3.09652387e+05  3.33404812e+05  3.09375800e+05 -3.47342632e+05
  1.42144073e+04  4.75584543e+04  1.54389422e+05 -1.05313675e+05
 -8.75901897e+04 -1.22727711e+05  1.63713703e+05  2.05225265e+05
 -1.37528961e+05 -3.66564517e+04 -1.77963175e+05  9.68933187e+04
 -5.50263285e+04  5.50263285e+04 -2.69344858e+04 -1.06530184e+05
 -5.06109044e+04 -3.90634827e+04  5.50263285e+04 -3.40397769e+04
  6.02005465e+03 -1.45829665e+05 -7.86539602e+04  3.92045347e+03
  6.48446806e+04 -4.43076458e+02 -5.50263285e+04  1.85101788e+05
  1.30075460e+05  2.80170732e+01  1.68701456e+02  0.00000000e+00
 -2.69344858e+04  4.43169771e+04 -9.05279588e+03 -3.90634827e+04
  5.50263285e+04 -3.40397769e+04 -1.45829665e+05 -7.86539602e+04
  3.92045347e+03  6.48446806e+04]
5073601.359277787


In [33]:
print('-'*20 + 'model coefficients' + '-'*20)
print((' '*24 + 'Intercept: {:>10.2f}').format(best_model.intercept_))
for feature, coef in zip(best_features, best_model.coef_):
  print((' '*(40-len(feature))+feature+': {:>10.2f}').format(coef))
print(f'{len(best_model.coef_)} features')

--------------------model coefficients--------------------
                        Intercept: 5073601.36
            BUILDING_CLASS_AT_PRESENT_A6: -309652.39
            BUILDING_CLASS_AT_PRESENT_A2:  333404.81
   NEIGHBORHOOD_SO. JAMAICA-BAISLEY PARK:  309375.80
               TAX_CLASS_AT_TIME_OF_SALE: -347342.63
                  NEIGHBORHOOD_LAURELTON:   14214.41
       BUILDING_CLASS_AT_TIME_OF_SALE_A3:   47558.45
           NEIGHBORHOOD_SOUTH OZONE PARK:  154389.42
                    NEIGHBORHOOD_BAYSIDE: -105313.67
       BUILDING_CLASS_AT_TIME_OF_SALE_A4:  -87590.19
       BUILDING_CLASS_AT_TIME_OF_SALE_A9: -122727.71
                        LAND_SQUARE_FEET:  163713.70
       BUILDING_CLASS_AT_TIME_OF_SALE_A5:  205225.27
            BUILDING_CLASS_AT_PRESENT_A9: -137528.96
                        COMMERCIAL_UNITS:  -36656.45
            BUILDING_CLASS_AT_PRESENT_S1: -177963.17
                               BOROUGH_1:   96893.32
            BUILDING_CLASS_AT_PRESENT_A4:  -550